# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from sqlalchemy import create_engine

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix, precision_score, accuracy_score, recall_score, f1_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
import random

from workspace_utils import active_session

from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import KNeighborsClassifier

import pickle

#%matplotlib inline
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///Database.db')
df = pd.read_sql_table('table', engine)


In [3]:
#df.related.unique()

In [4]:
#df.related.replace(2,1,inplace=True)

In [8]:
X = df['message']
Y = df.iloc[:,4:]

In [9]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [5]:
X.shape

(26216,)

In [6]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X.shape

(26216,)

In [8]:
#category_names = Y.columns
#category_names

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    # normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = text.split()
    
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    #stop_words = stopwords.words("english")
    
    # get tokens from text
    #tokens = word_tokenize(words)
    #lemmatizer = WordNetLemmatizer()
    
    # Reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in words]
 
    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in stemmed]
    
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [11]:
# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 20972 samples.
Testing set has 5244 samples.


In [12]:
Y_train.shape

(20972, 36)

In [12]:
X_train.shape

(20972,)

In [13]:
Y_test.shape

(5244, 36)

In [13]:
X_test.shape

(5244,)

AttributeError: 'DataFrame' object has no attribute 'type'

In [14]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
# train classifier
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [16]:
# predict on test data
Y_pred = pipeline.predict(X_test)

In [17]:
# convert to dataframe
Y_pred = pd.DataFrame (Y_pred, columns = Y_test.columns)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
for column in Y.columns:
    print('\n---- {} ----\n{}\n'.format(column, classification_report(Y_test[column], Y_pred[column])))



---- related ----
             precision    recall  f1-score   support

          0       0.63      0.45      0.52      1217
          1       0.85      0.92      0.88      3996
          2       0.28      0.35      0.31        31

avg / total       0.79      0.80      0.79      5244



---- request ----
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4333
          1       0.79      0.42      0.55       911

avg / total       0.87      0.88      0.86      5244



---- offer ----
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5229
          1       0.00      0.00      0.00        15

avg / total       0.99      1.00      1.00      5244



---- aid_related ----
             precision    recall  f1-score   support

          0       0.76      0.86      0.81      3042
          1       0.76      0.61      0.68      2202

avg / total       0.76      0.76      0.75      5244



----

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [19]:
# accuracy
#print ('Accuracy: {}'.format(accuracy_score(Y_test.iloc[:, 1:], Y_pred.iloc[:, 1:])))

In [20]:
#print(classification_report(Y_test.iloc[:,1:].values, np.array([x[1:] for x in Y_pred]), target_names=Y.columns))

In [21]:
# classification report
#for column in Y_test.columns:
#    print('FEATURE: {}\n'.format(column), classification_report(Y_test[column],Y_pred[column]))

### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [25]:
parameters = {
#        'vect__ngram_range': ((1, 1), (1, 2)),
#        'vect__max_df': (0.5, 0.75, 1.0),
#        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
#        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3],
    }
#scorer = make_scorer(precision_score)

cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 2) 

In [26]:
with active_session():
    
    cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True ........
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, total= 2.3min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True ........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, total= 2.3min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True ........
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, total= 2.3min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=False .......
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=False, total= 2.3min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=False .......
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=False, total= 2.3min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=False .......
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=False, total= 2.3min
[CV] clf__estimator__min_samples_split=3, tfidf__use_idf=True ........
[CV]  clf__estimator__min_samples_split=3, tfidf__use_idf=True, total= 2.2min
[CV] clf__estimator__min_samples_split=3, tfidf__use_idf=True ........
[CV]  clf__estimator__min_samples_split=3, tfidf__use_idf=True, total= 2.2min
[CV] clf__estimator__min_

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 41.2min finished


In [27]:
cv.best_params_


{'clf__estimator__min_samples_split': 2, 'tfidf__use_idf': True}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
Y_tuned_pred = cv.predict(X_test)

In [29]:
# convert to dataframe
Y_tuned_pred = pd.DataFrame (Y_tuned_pred, columns = Y_test.columns)

In [30]:
for column in Y.columns:
    print('\n---- {} ----\n{}\n'.format(column, classification_report(Y_test[column], Y_tuned_pred[column])))


---- related ----
             precision    recall  f1-score   support

          0       0.62      0.46      0.53      1217
          1       0.85      0.91      0.88      3996
          2       0.28      0.42      0.34        31

avg / total       0.79      0.80      0.79      5244



---- request ----
             precision    recall  f1-score   support

          0       0.89      0.98      0.94      4333
          1       0.83      0.45      0.58       911

avg / total       0.88      0.89      0.87      5244



---- offer ----
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5229
          1       0.00      0.00      0.00        15

avg / total       0.99      1.00      1.00      5244



---- aid_related ----
             precision    recall  f1-score   support

          0       0.75      0.87      0.80      3042
          1       0.77      0.60      0.67      2202

avg / total       0.76      0.76      0.75      5244



----

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
def display_results(cv, Y_test, Y_pred):
    labels = np.unique(Y_pred)
    #predictions = Y_pred

    #cm = confusion_matrix(labels, predictions)
    #accuracy = 
    #recall = np.diag(cm) / np.sum(cm, axis = 1)
    #precision = np.diag(cm) / np.sum(cm, axis = 0)
    ##confusion_mat = confusion_matrix(np.array(Y_test), Y_pred, labels=labels)
    ##accuracy = (Y_pred == Y_test).mean()

    #print("Recall score:", recall)
    #print("Confusion Matrix:\n", cm)
    ##print("Accuracy:", accuracy)
    #print("\nPrecision:", precision)
    results = []
    
    for i in range(len(labels)):
        accuracy = accuracy_score(Y_test.iloc[:, i].values, Y_pred.iloc[:, i].values)
       # accuracy_score(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]
        precision = precision_score(Y_test.iloc[:, i].values, Y_pred.iloc[:, i].values, average='weighted')
        recall = recall_score(Y_test.iloc[:, i].values, Y_pred.iloc[:, i].values, average='weighted')
        f1 = f1_score(Y_test.iloc[:, i].values, Y_pred.iloc[:, i].values, average='weighted')
        
        results.append([accuracy, precision, recall, f1])
    
    # Print out metrics
   # print("Labels:", labels)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("f1 score:", f1)

In [32]:
display_results(cv, Y_test, Y_tuned_pred)

Accuracy: 0.997139588101
Precision: 0.994287358158
f1 score: 0.99571143057


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
# try KNN
pipelineKNN = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

In [10]:
pipelineKNN.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__leaf_size', 'clf__estimator__metric', 'clf__estimator__metric_params', 'clf__estimator__n_jobs', 'clf__estimator__n_neighbors', 'clf__estimator__p', 'clf__estimator__weights', 'clf__estimator', 'clf__n_jobs'])

In [11]:
parametersKNN = {
#         'vect__max_df': (0.5, 1.0),
#         'vect__max_features': (None, 5000, 10000),
         'tfidf__use_idf': [True],
         'clf__estimator__n_neighbors': [3, 4, 5],
    }

cvKNN = GridSearchCV(pipelineKNN, param_grid = parametersKNN, verbose = 2) 

In [12]:
with active_session():
    cvKNN.fit(X_train, Y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] clf__estimator__n_neighbors=3, tfidf__use_idf=True ..............
[CV]  clf__estimator__n_neighbors=3, tfidf__use_idf=True, total= 3.6min
[CV] clf__estimator__n_neighbors=3, tfidf__use_idf=True ..............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.3min remaining:    0.0s


[CV]  clf__estimator__n_neighbors=3, tfidf__use_idf=True, total= 3.4min
[CV] clf__estimator__n_neighbors=3, tfidf__use_idf=True ..............
[CV]  clf__estimator__n_neighbors=3, tfidf__use_idf=True, total= 3.4min
[CV] clf__estimator__n_neighbors=4, tfidf__use_idf=True ..............
[CV]  clf__estimator__n_neighbors=4, tfidf__use_idf=True, total= 3.7min
[CV] clf__estimator__n_neighbors=4, tfidf__use_idf=True ..............
[CV]  clf__estimator__n_neighbors=4, tfidf__use_idf=True, total= 3.7min
[CV] clf__estimator__n_neighbors=4, tfidf__use_idf=True ..............
[CV]  clf__estimator__n_neighbors=4, tfidf__use_idf=True, total= 3.7min
[CV] clf__estimator__n_neighbors=5, tfidf__use_idf=True ..............
[CV]  clf__estimator__n_neighbors=5, tfidf__use_idf=True, total= 3.7min
[CV] clf__estimator__n_neighbors=5, tfidf__use_idf=True ..............
[CV]  clf__estimator__n_neighbors=5, tfidf__use_idf=True, total= 3.7min
[CV] clf__estimator__n_neighbors=5, tfidf__use_idf=True ..............

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 77.8min finished


In [13]:
cvKNN.best_params_

{'clf__estimator__n_neighbors': 5, 'tfidf__use_idf': True}

In [16]:
Y_KNN_pred = cvKNN.predict(X_test)

# convert to dataframe
Y_KNN_pred = pd.DataFrame (Y_KNN_pred, columns = Y_test.columns)

# classification report
for column in Y.columns:
    print('\n---- {} ----\n{}\n'.format(column, classification_report(Y_test[column], Y_KNN_pred[column])))


---- related ----
             precision    recall  f1-score   support

          0       0.59      0.34      0.43      1217
          1       0.82      0.92      0.87      3996
          2       0.24      0.39      0.29        31

avg / total       0.76      0.78      0.76      5244



---- request ----
             precision    recall  f1-score   support

          0       0.84      1.00      0.91      4333
          1       0.79      0.08      0.15       911

avg / total       0.83      0.84      0.78      5244



---- offer ----
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5229
          1       0.00      0.00      0.00        15

avg / total       0.99      1.00      1.00      5244



---- aid_related ----
             precision    recall  f1-score   support

          0       0.59      0.99      0.74      3042
          1       0.74      0.04      0.07      2202

avg / total       0.65      0.59      0.46      5244



----

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
display_results(cvKNN, Y_test, Y_KNN_pred)

Accuracy: 0.997139588101
Precision: 0.994287358158
f1 score: 0.99571143057


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# StartingVerbExtractor Class Definition
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 9. Export your model as a pickle file

In [18]:
pickle.dump(cv, open('classifier.p', 'wb'))

NameError: name 'cv' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.